In [53]:
from astropy.table import Table, Column, Row
import random
import time
from datetime import date
import calendar
import numpy as np

First I read in my weather stats table as provided by Jess

In [54]:
wdata = Table.read('data/weather-average.csv', format='csv')

I then make new columns containing the cumulative fraction of time in each weather band (for use later). 

In [55]:
CumulativeBand1 = Column(wdata['Band1Percentage'], name='CumulativeBand1', format='.1f')
wdata.add_column(CumulativeBand1)
CumulativeBand2 = Column([x['CumulativeBand1']+x['Band2Percentage'] for x in wdata], name='CumulativeBand2', format='.1f')
wdata.add_column(CumulativeBand2)
CumulativeBand3 = Column([x['CumulativeBand2']+x['Band3Percentage'] for x in wdata], name='CumulativeBand3', format='.1f')
wdata.add_column(CumulativeBand3)
CumulativeBand4 = Column([x['CumulativeBand3']+x['Band4Percentage'] for x in wdata], name='CumulativeBand4', format='.1f')
wdata.add_column(CumulativeBand4)
CumulativeBand5 = Column([x['CumulativeBand4']+x['Band5Percentage'] for x in wdata], name='CumulativeBand5', format='.1f')
wdata.add_column(CumulativeBand5)

In [56]:
#wdata

I define a loook up dictionary for easy table access - months are not month values but rather positions in the table


In [57]:
dict = {'February': 0, 'March': 1, 'April': 2, 'May': 3, 'June': 4, 'July': 5, 'August': 6, 'September': 7, 'October': 8, 'November': 9, 'December': 10, 'January': 11}

Now I need to load in my schedule file from this it will detemrine the month and number of days I need to predict for

In [59]:
# need to supply table with list of dates on which we will observe - 
# from this table my code needs to know month and number of days - to allocated a weather band for the night
# the UT date and weather bad is then needed as input into Michael Polhen's code

sdata = Table.read('data/18A_schedule.csv', format='csv')

# splitting up YYYYMMDD into YYYY MM DD
yearForTable = Column([int(x/10000) for x in sdata['Date']], name='Year')
monthForTable = Column([calendar.month_name[ int(int(x-((int(x/10000))*10000))/100)] for x in sdata['Date']], name='Month')
dayForTable = Column([x - (int(x/100)*100) for x in sdata['Date']], name='Day')
sdata.add_column(yearForTable)
sdata.add_column(monthForTable)
sdata.add_column(dayForTable)

# obtaines data when not PI time (PI = UH and DDT and E&C)

LAPsdata = sdata[sdata['Priority1']!='PI'] 

# for now only look at one month

month = 'February' #### CHANGE

MonthLAPSdata = LAPsdata[LAPsdata['Month']==month]

numberofdays = (len(MonthLAPSdata))

MonthLAPSdata

Date,Priority1,Priority2,Year,Month,Day
int64,str8,str8,int64,str8,int64
20180201,M16AL006,M17BL009,2018,February,1
20180202,M16AL006,M17BL009,2018,February,2
20180203,M16AL006,M17BL009,2018,February,3
20180204,M16AL006,M17BL009,2018,February,4
20180205,M16AL006,M17BL009,2018,February,5
20180206,M17BL004,--,2018,February,6
20180207,M17BL004,--,2018,February,7
20180208,M17BL004,--,2018,February,8
20180209,M17BL004,--,2018,February,9


In [61]:
print ("examining {} with {} days scheduled".format(month,numberofdays))

examining February with 14 days scheduled


I then generate random numbers from 0-100 for the number of days dedicate to LAP programs in my month and use the "cumulative fraction of weather" stats to allocated an entire night to a weather band

In [38]:
randomweather = random.sample(range(1, 100),numberofdays)
print ("Examining {} nights alloacted in {} \n".format(numberofdays,month))

print (("fraction of nights Band 1={}%, 2={}%, 3={}%, 4={}% and 5={}%\n".format(wdata['Band1Percentage'][dict[month]],wdata['Band2Percentage'][dict[month]],wdata['Band3Percentage'][dict[month]],wdata['Band4Percentage'][dict[month]],wdata['Band5Percentage'][dict[month]])))

b1 = 0 # counter for number of days in band 1
b2 = 0
b3 = 0
b4 = 0
b5 = 0
b6 = 0
sdatabanddata  = ([])

for x in randomweather:
#    print (x)
    if x <= wdata['CumulativeBand1'][dict[month]]:
        print ("Band 1 night")
        b1 += 1
        sdatabanddata.append(1)
    if wdata['CumulativeBand1'][dict[month]] <= x <= wdata['CumulativeBand2'][dict[month]]: 
        print ("Band 2 night")
        b2 += 1
        sdatabanddata.append(2)
    if wdata['CumulativeBand2'][dict[month]] <= x <= wdata['CumulativeBand3'][dict[month]]:  
        print ("Band 3 night")
        b3 += 1
        sdatabanddata.append(3)
    if wdata['CumulativeBand3'][dict[month]] <= x <= wdata['CumulativeBand4'][dict[month]]:  
        print ("Band 4 night")
        b4 += 1
        sdatabanddata.append(4)
    if wdata['CumulativeBand4'][dict[month]] <= x <= wdata['CumulativeBand5'][dict[month]]: 
        print ("Band 5 night")
        b5 += 1
        sdatabanddata.append(5)
    if x >= wdata['CumulativeBand5'][dict[month]]:
        print ("Closed")
        b6 += 1
        sdatabanddata.append(6)
            
print ("\nfraction output 1={:.2f}%, 2={:.2f}%, 3={:.2f}%, 4={:.2f}% and 5={:.2f}% (closed {:.2f}%)".format(b1/numberofdays*100,b2/numberofdays*100,b3/numberofdays*100,b4/numberofdays*100,b5/numberofdays*100,b6/numberofdays*100))
print (sdatabanddata)

Examining 14 nights alloacted in February 

fraction of nights Band 1=15.7%, 2=23.3%, 3=16.0%, 4=20.7% and 5=17.8%

Band 5 night
Band 3 night
Band 5 night
Band 2 night
Band 5 night
Band 5 night
Band 5 night
Band 4 night
Band 2 night
Band 5 night
Band 5 night
Band 4 night
Band 3 night
Band 2 night

fraction output 1=0.00%, 2=21.43%, 3=14.29%, 4=14.29% and 5=50.00% (closed 0.00%)
[5, 3, 5, 2, 5, 5, 5, 4, 2, 5, 5, 4, 3, 2]


In [ ]:
#sdatabanddata.T
#print (sdatabanddata.T)
#banddata = Column(sdatabanddata, name='WeatherBand')
#sdata.add_column(banddata)


In essence I then need to calculate how many night in a month are in each weatherbands and run Michael's code on the relevent 